In [10]:
%cd "c:/Kuliah ITS Farhan/Semester 3/A_Final_Project/FP_KK/fp_kk_11_early_warning_hoax"


c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\FP_KK\fp_kk_11_early_warning_hoax


In [11]:
import os
print(os.getcwd())
print(os.listdir())


c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\FP_KK\fp_kk_11_early_warning_hoax
['.env', '.git', '.gitignore', '.venv', 'app', 'data', 'notebooks', 'README.md', 'requirements.txt', 'src']


In [13]:
import numpy as np
import pandas as pd

from src.fuzzy_system import hitung_resiko_hoaks

# pakai nama file yang benar
DATA_PATH = "data/processed/hoax_umpo_sample_100_llm.csv"

df_train = pd.read_csv(DATA_PATH)

print(df_train.shape)
df_train.head()


(100, 7)


,sample_id,text,label_raw,label_resiko_hoaks,intensitas_emosi,kecurigaan_format,kredibilitas_rendah
0,1,"Ngeliat harimau yang endut gitu, jadi keinget ...",hoax,100,0.65,0.45,0.65
1,2,Puasa Puasa RT \n@hunterisma\n: enak tuh ayam ...,hoax,100,0.10,0.00,0.10
2,3,Viral!!! Mohon Sebarkan Garam Mengandung Pecah...,hoax,100,0.90,0.95,0.90
3,4,"Pake aja kalung mengkudu, terhindar dari penya...",hoax,100,0.40,0.70,0.90
4,5,"#Share Ternyata, Daun dan Akar Dandelion Bersi...",hoax,100,0.70,0.60,0.80


In [15]:
# pakai df_train yang sudah kamu load
df_train = df_train.rename(columns={"label_resiko_hoaks": "resiko_target"})
df_train.head()


,sample_id,text,label_raw,resiko_target,intensitas_emosi,kecurigaan_format,kredibilitas_rendah
0,1,"Ngeliat harimau yang endut gitu, jadi keinget ...",hoax,100,0.65,0.45,0.65
1,2,Puasa Puasa RT \n@hunterisma\n: enak tuh ayam ...,hoax,100,0.10,0.00,0.10
2,3,Viral!!! Mohon Sebarkan Garam Mengandung Pecah...,hoax,100,0.90,0.95,0.90
3,4,"Pake aja kalung mengkudu, terhindar dari penya...",hoax,100,0.40,0.70,0.90
4,5,"#Share Ternyata, Daun dan Akar Dandelion Bersi...",hoax,100,0.70,0.60,0.80


In [16]:
import numpy as np

from src.fuzzy_system import hitung_resiko_hoaks

def hitung_mae_fuzzy_scaled(weights, df, target_col="resiko_target",
                            verbose=False, n_sample=3):
    """
    weights: [w_emosi, w_format, w_kred]
    df: DataFrame dengan 3 skor LLM + kolom target
    """
    w = np.array(weights, dtype=float)
    targets = df[target_col].values
    preds = []

    if verbose:
        print("\n=== DETAIL PERHITUNGAN (contoh beberapa baris) ===")
        print("w =", w)

    for idx, (_, row) in enumerate(df.iterrows()):
        e_raw = row["intensitas_emosi"]
        f_raw = row["kecurigaan_format"]
        k_raw = row["kredibilitas_rendah"]

        # scaling + clamp [0,1]
        e_scaled = np.clip(e_raw * w[0], 0.0, 1.0)
        f_scaled = np.clip(f_raw * w[1], 0.0, 1.0)
        k_scaled = np.clip(k_raw * w[2], 0.0, 1.0)

        # fuzzy Mamdani (rule 10 terbaru)
        y_pred = hitung_resiko_hoaks(
            intensitas_emosi=e_scaled,
            kecurigaan_format=f_scaled,
            kredibilitas_rendah=k_scaled,
            output_scale_100=True,
        )
        preds.append(y_pred)

        if verbose and idx < n_sample:
            print(f"\n--- Data index {idx} (sample_id={row['sample_id']}) ---")
            print(f"  skor LLM  : emosi={e_raw:.3f}, format={f_raw:.3f}, kred={k_raw:.3f}")
            print(f"  setelah w : emosi'={e_scaled:.3f}, format'={f_scaled:.3f}, kred'={k_scaled:.3f}")
            print(f"  fuzzy out : {y_pred:.3f}")
            print(f"  target    : {targets[idx]:.3f}")
            print(f"  |error|   : {abs(y_pred - targets[idx]):.3f}")

    preds = np.array(preds)
    mae = np.mean(np.abs(preds - targets))

    if verbose:
        print("\nMAE total (berdasarkan seluruh data):", mae)

    return mae


In [17]:
w_lama = [0.5, 1.5, 1.33333334]
mae_lama = hitung_mae_fuzzy_scaled(w_lama, df_train, verbose=True, n_sample=2)
print("\nMAE dengan bobot lama:", mae_lama)



=== DETAIL PERHITUNGAN (contoh beberapa baris) ===
w = [0.5        1.5        1.33333334]

--- Data index 0 (sample_id=1) ---
  skor LLM  : emosi=0.650, format=0.450, kred=0.650
  setelah w : emosi'=0.325, format'=0.675, kred'=0.867
  fuzzy out : 81.936
  target    : 100.000
  |error|   : 18.064

--- Data index 1 (sample_id=2) ---
  skor LLM  : emosi=0.100, format=0.000, kred=0.100
  setelah w : emosi'=0.050, format'=0.000, kred'=0.133
  fuzzy out : 14.309
  target    : 100.000
  |error|   : 85.691

MAE total (berdasarkan seluruh data): 40.14838005484544

MAE dengan bobot lama: 40.14838005484544


In [18]:
def run_pso_verbose(
    df,
    n_particles=20,
    n_iters=50,
    bounds=(0.5, 1.5),
    seed=42,
    print_every=10,
):
    """
    PSO untuk mencari [w_emosi, w_format, w_kred] dengan log yang cukup detail.
    """
    np.random.seed(seed)

    dim = 3  # w_emosi, w_format, w_kred
    low, high = bounds

    positions = np.random.uniform(low, high, size=(n_particles, dim))
    velocities = np.zeros_like(positions)

    # fitness awal
    fitness = np.array([hitung_mae_fuzzy_scaled(w, df) for w in positions])

    pbest_pos = positions.copy()
    pbest_fit = fitness.copy()

    gbest_idx = np.argmin(pbest_fit)
    gbest_pos = pbest_pos[gbest_idx].copy()
    gbest_fit = float(pbest_fit[gbest_idx])

    print("=== INIT PSO ===")
    print(f"Jumlah partikel : {n_particles}")
    print(f"Iterasi maksimum: {n_iters}")
    print(f"Bounds w        : {bounds}")
    print(f"Best awal       : MAE={gbest_fit:.4f}, w={gbest_pos}\n")

    w_inertia = 0.7
    c1 = 1.5
    c2 = 1.5

    history = []

    for it in range(n_iters):
        for i in range(n_particles):
            r1, r2 = np.random.rand(), np.random.rand()

            velocities[i] = (
                w_inertia * velocities[i]
                + c1 * r1 * (pbest_pos[i] - positions[i])
                + c2 * r2 * (gbest_pos - positions[i])
            )
            positions[i] += velocities[i]
            positions[i] = np.clip(positions[i], low, high)

            fit_i = hitung_mae_fuzzy_scaled(positions[i], df)
            fitness[i] = fit_i

            if fit_i < pbest_fit[i]:
                pbest_fit[i] = fit_i
                pbest_pos[i] = positions[i].copy()

        gbest_idx = np.argmin(pbest_fit)
        if pbest_fit[gbest_idx] < gbest_fit:
            gbest_fit = float(pbest_fit[gbest_idx])
            gbest_pos = pbest_pos[gbest_idx].copy()

        history.append(gbest_fit)

        if (it % print_every == 0) or (it == n_iters - 1):
            print(f"[Iter {it:03d}] MAE terbaik: {gbest_fit:.4f}, w={gbest_pos}")
            print("  → Detail perhitungan dengan w terbaik saat ini:")
            _ = hitung_mae_fuzzy_scaled(gbest_pos, df, verbose=True, n_sample=2)
            print("-" * 60)

    print("\n=== PSO SELESAI ===")
    print(f"Best MAE : {gbest_fit:.4f}")
    print(f"Best w   : {gbest_pos}")

    return {
        "best_position": gbest_pos,
        "best_fitness": gbest_fit,
        "history": history,
    }


In [19]:
result = run_pso_verbose(
    df=df_train,
    n_particles=20,
    n_iters=50,       # kalau terasa masih goyang, boleh naikin ke 100
    bounds=(0.5, 1.5),
    seed=42,
    print_every=10,
)

best_w = result["best_position"]
best_mae = result["best_fitness"]

print("\n=== HASIL AKHIR PSO (VERSI BARU) ===")
print("w_emosi , w_format , w_kred =", best_w)
print("MAE terbaik                =", best_mae)


=== INIT PSO ===
Jumlah partikel : 20
Iterasi maksimum: 50
Bounds w        : (0.5, 1.5)
Best awal       : MAE=36.6359, w=[0.53438852 1.4093204  0.75877998]

[Iter 000] MAE terbaik: 36.6040, w=[0.52456004 1.46703297 0.74871992]
  → Detail perhitungan dengan w terbaik saat ini:

=== DETAIL PERHITUNGAN (contoh beberapa baris) ===
w = [0.52456004 1.46703297 0.74871992]

--- Data index 0 (sample_id=1) ---
  skor LLM  : emosi=0.650, format=0.450, kred=0.650
  setelah w : emosi'=0.341, format'=0.660, kred'=0.487
  fuzzy out : 55.311
  target    : 100.000
  |error|   : 44.689

--- Data index 1 (sample_id=2) ---
  skor LLM  : emosi=0.100, format=0.000, kred=0.100
  setelah w : emosi'=0.052, format'=0.000, kred'=0.075
  fuzzy out : 13.582
  target    : 100.000
  |error|   : 86.418

MAE total (berdasarkan seluruh data): 36.60404117697392
------------------------------------------------------------
[Iter 010] MAE terbaik: 35.6720, w=[0.53247573 1.5        0.5       ]
  → Detail perhitungan dengan 